Deep Learning based Medical Diagnoser using LSTM

- We will build a deep learning model that will be trained on Patient's Problems which will be textual data,
then our model will give the predicted Disease and will recommend Medicine to treat the patient's problem as an output.

- This is clearly an application of Recurrent Neural Network (RNN).
This is because we need a model that will store the information from the previous text and use it later to predict the output.
Hence, we will use the Long Short-Term Memory (LSTM) algorithm with Tensorflow to train our model.

- Long Short-Term Memory (LSTM) Networks

  - When dealing with textual data, such as patient symptoms, a specific type of deep learning architecture called a Long Short-Term Memory (LSTM) network is often used.
  - LSTM networks are well-suited for tasks involving sequences of data, as they can learn long-term dependencies between elements in the sequence.

  - For example, consider a patient describing their symptoms as " I've experienced a loss of appetite and don't enjoy food anymore,
  followed by fatigue and muscle weakness." An LSTM network can understand the importance of the order of these symptoms ("loss of appetite" followed by "fatigue and muscle weakness")
  to make an accurate diagnosis.

- The dataset includes:
  - Patient Symptoms: Textual descriptions of the patient's symptoms.
  - Diagnoses: The confirmed diseases for each patient.
  - Medications: The prescribed medications for each patient's condition

- The model architecture utilizes an LSTM layer to process the sequence of tokens representing the patient's symptoms. The output of the LSTM layer is then fed into two separate dense layers, one for predicting the disease and another for predicting the medication.

Importing Libraries:
  - 'Tokenizer' from tensorflow will be used for text tokenization
  - 'pad_sequences' will be used for sequence padding.
  - 'to_categorical' will be used for converting labels to binary class matrices
  - 'LabelEncoder' from scikit-learn will be used for encoding text labels as integers.

In [1]:
# !pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

Loading the Dataset

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/adil200/Medical-Diagnoser/main/medical_data.csv')
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


Data Processing and Preparation
  - Text Tokenization: Converting textual data into sequences of numbers that the model can process.
  - Padding Sequences: Making all sequences the same length by adding padding characters at the beginning or end of shorter sequences.
  - Label Encoding: Converting categorical variables, such as disease names and medication names, into numerical labels.

- Tokenizing and Sequencing Text Data

In [4]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Patient_Problem'])

sequences = tokenizer.texts_to_sequences(data['Patient_Problem'])

  - A 'tokenizer' variable is created to convert the textual data into sequences of integers. It only considers the top 5,000 words in the dataset in order to reduce the complexity. If the model encounters any out-of-vocabulary words during the training process then it will be replaced with the '<OOV>' token.

- Padding Sequences
- In order to make the input sequences have the same length, the code finds the longest sequence and pads all other sequences with zeros at the end ('post' padding) to match this sentence.

In [5]:
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

- Encoding the Labels and Converting them to Categorical
- We will encode the 'Disease' and 'Prescription' columns as integers. Then the integer-encoded labels are converted into binary class matrices.

In [6]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(data['Disease'])
prescription_labels = label_encoder_prescription.fit_transform(data['Prescription'])

# Converting labels to categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)

- Combining Labels into a Multi-label Target Variable
- Finally, now we will stack the binary class matrices together to form a single multi-label target variable 'Y'. This allows the model to predict both 'Disease' and 'Prescription' from the patient's problem.

In [7]:
Y = np.hstack((disease_labels_categorical, prescription_labels_categorical))

Model Building
  - Now, we will build the model using the LSTM and Sequential algorithm from TensorFlow. This model will learn from our preprocessed dataset to predict diseases based on patient symptoms.



- Defining Model Architecture
- We will use the 'Model' and 'Input' to define the model architecture, and 'Embedding' to convert the integer sequences into dense vectors of fixed size.
- We will use 'Dense' for output layers that make predictions.

In [8]:
input_layer = Input(shape=(max_length,))

embedding = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax', name='disease_output')(lstm_layer)
prescription_output = Dense(len(label_encoder_prescription.classes_), activation='softmax', name='prescription_output')(lstm_layer)

- The model firstly have, an input layer that can handle sequences up to a certain length. Then there's an embedding layer that turns the numbers into vectors. After that, there's an LSTM layer that looks at the order of things, and finally, two dense layers that predict diseases and prescriptions using a softmax function for classification.

Compiling the model

In [9]:
model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])
model.compile(
    loss={'disease_output': 'categorical_crossentropy',
    'prescription_output': 'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 17)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 17, 64)    │    320,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     33,024 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_output      │ (None, 178)       │     11,570 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prescription_output │ (None, 388)       │     25,220 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 389,814 (1.49 MB)

 Trainable params: 389,814 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

- Training the model

In [10]:
model.fit(padded_sequences,
    {'disease_output': disease_labels_categorical,
    'prescription_output': prescription_labels_categorical},
    epochs=100,
    batch_size=32
)

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - disease_output_accuracy: 0.0080 - disease_output_loss: 5.1807 - loss: 11.1441 - prescription_output_accuracy: 0.0026 - prescription_output_loss: 5.9635
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - disease_output_accuracy: 0.0347 - disease_output_loss: 5.1619 - loss: 11.1225 - prescription_output_accuracy: 0.0018 - prescription_output_loss: 5.9605
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - disease_output_accuracy: 0.0277 - disease_output_loss: 5.1105 - loss: 11.0723 - prescription_output_accuracy: 0.0129 - prescription_output_loss: 5.9614
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - disease_output_accuracy: 0.0282 - disease_output_loss: 4.9767 - loss: 10.9421 - prescription_output_accuracy: 0.0129 - prescription_output_loss: 5.9657
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - disease_output_accuracy: 0.0371 - disease_output_loss: 4.8612 - loss: 10.7903 - prescription_output_accuracy: 0.0195 - prescri

Making Predictions

The model is used to make predictions for new patients:

1. Pre-processed the patient's symptoms by performing tokenization and padding.
2. Feed the pre-processed data into the trained model.
3. The model predicts the disease and medication based on the patient's symptoms.
4. The predicted disease and medication will be presented.

In [11]:
def make_prediction(patient_problem):
  # Preprocessing the input
  sequence = tokenizer.texts_to_sequences([patient_problem])
  padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

  # Making the prediction
  predictions = model.predict(padded_sequence)

  # Decoding the sequence
  disease_index = np.argmax(predictions[0], axis=1)[0]
  prediction_index = np.argmax(predictions[1], axis=1)[0]

  disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
  prescription_predicted = label_encoder_prescription.inverse_transform([prediction_index])[0]

  print(f"Predicted Disease: {disease_predicted}")
  print(f"Suggested Prescription: {prescription_predicted}")

patient_input = "I've experienced a loss of appetite and don't enjoy food anymore."
make_prediction(patient_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Predicted Disease: Depression
Suggested Prescription: Antidepressants; eating nutrient-rich foods.


Conclusion

- Adding deep learning to medical diagnostics is a game-changer in healthcare.The model is trained on the preprocessed dataset, iteratively adjusting its internal parameters to improve its accuracy in predicting diseases and medications based on patient symptoms.

- In addition, these fancy models can find new patterns in big datasets, which could help us better understand tricky medical conditions. The future looks bright as we use AI to bring together technology and human knowledge for the benefit of patient health and medical research.